In [ ]:
! pip3 install langchain langchain-community langchain-text-splitters deepeval python-dotenv pandas
# chromadb tiktoken boto3 aiobotocore botocore 

In [2]:
#import boto3
from dotenv import load_dotenv, find_dotenv
import os
from deepeval.models import GPTModel
import re
# import random

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
# # === AWS Configuration === #
# COGNITO_REGION = os.getenv("COGNITO_REGION")
# BEDROCK_REGION = os.getenv("BEDROCK_REGION")
# MODEL_ID1 = os.getenv("MODEL_ID1")
# MODEL_ID2 = os.getenv("MODEL_ID2")
# IDENTITY_POOL_ID = os.getenv("IDENTITY_POOL_ID")
# USER_POOL_ID = os.getenv("USER_POOL_ID")
# APP_CLIENT_ID = os.getenv("APP_CLIENT_ID")
# USERNAME = os.getenv("USERNAME")
# PASSWORD = os.getenv("PASSWORD")

In [5]:
# # === Helper: Get AWS Credentials === #
# def get_credentials(username, password):
#     idp_client = boto3.client("cognito-idp", region_name=COGNITO_REGION)
#     response = idp_client.initiate_auth(
#         AuthFlow="USER_PASSWORD_AUTH",
#         AuthParameters={"USERNAME": username, "PASSWORD": password},
#         ClientId=APP_CLIENT_ID,
#     )
#     id_token = response["AuthenticationResult"]["IdToken"]

#     identity_client = boto3.client("cognito-identity", region_name=COGNITO_REGION)
#     identity_response = identity_client.get_id(
#         IdentityPoolId=IDENTITY_POOL_ID,
#         Logins={f"cognito-idp.{COGNITO_REGION}.amazonaws.com/{USER_POOL_ID}": id_token},
#     )

#     creds_response = identity_client.get_credentials_for_identity(
#         IdentityId=identity_response["IdentityId"],
#         Logins={f"cognito-idp.{COGNITO_REGION}.amazonaws.com/{USER_POOL_ID}": id_token},
#     )

#     return creds_response["Credentials"]

In [6]:
# credentials = get_credentials(USERNAME, PASSWORD)

In [7]:
# os.environ["AWS_ACCESS_KEY_ID"]=credentials.get('AccessKeyId')
# os.environ["AWS_SECRET_ACCESS_KEY"]=credentials.get('SecretKey')
# os.environ["AWS_SESSION_TOKEN"]=credentials.get('SessionToken')

In [8]:
# llm = AmazonBedrockModel(
#     model_id=MODEL_ID1,
#     region_name=BEDROCK_REGION,
# )

llm = GPTModel(model="o4-mini", temperature=1.0)

In [9]:
llm.model

In [10]:
llm.model_name

'o4-mini'

In [11]:
# llm.generation_kwargs = {
#     "maxTokens": 200,
#     "temperature":0
# }

In [12]:
# from deepeval.models import OllamaEmbeddingModel

In [13]:
# emb_model = OllamaEmbeddingModel(model="bge-m3:latest")

In [14]:
# emb_model.model_name = "bge-m3:latest"

In [15]:
# emb_model.model_name

In [16]:
#from deepeval.synthesizer.config import ContextConstructionConfig
from deepeval.synthesizer.config import FiltrationConfig
from deepeval.synthesizer.config import EvolutionConfig
from deepeval.synthesizer import Evolution
from deepeval.synthesizer.config import StylingConfig
from deepeval.synthesizer import Synthesizer

In [17]:
filtration_config = FiltrationConfig(
  synthetic_input_quality_threshold=0.8
)

In [18]:
evolution_config = EvolutionConfig(
    evolutions={
        Evolution.MULTICONTEXT: 1/2, # sticks to the context
        Evolution.CONCRETIZING: 1/2, # sticks to the context
        #Evolution.CONSTRAINED: 1/4, # sticks to the context
        #Evolution.COMPARATIVE: 1/4, # sticks to the context
    },
    num_evolutions=2
)

In [19]:
styling_config = StylingConfig(
  input_format="Questions in English regarding Canvas LMS navigation, usage and troubleshooting.",
  expected_output_format="Step-by-step instructions or consice responses (depending on the query) to help users understand, navigate, use and troubleshoot Canvas LMS.",
  task="Answering user queries about Canvas LMS and returning the results to users",
  scenario="Non-technical users, new to Canvas LMS, seeking help with navigation, usage and troubleshooting using plain English.",
)

In [20]:
synthesizer = Synthesizer(
    model=llm, 
    async_mode=False, 
    filtration_config=filtration_config, 
    evolution_config=evolution_config, 
    styling_config=styling_config,
    cost_tracking=True
)

In [21]:
canvas_basics_guide = [
    # General Information
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-Canvas/ta-p/45",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/Where-do-I-find-my-institution-s-URL-to-access-Canvas/ta-p/82",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-the-browser-and-computer-requirements-for-Instructure/ta-p/66",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-the-limited-support-guidelines-for-mobile-browsers-on/ta-p/389121",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-clear-my-browser-cache-and-cookies/ta-p/43",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/Can-I-repurpose-content-from-the-Guides-for-my-institution-s-use/ta-p/389122",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/Where-can-I-download-the-Guides-PDF-manuals/ta-p/389123",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-the-Canvas-accessibility-standards/ta-p/1564",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-reset-my-Canvas-password/ta-p/446128",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-allow-pop-ups-for-Canvas-in-my-browser/ta-p/33",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/Which-languages-does-Canvas-support/ta-p/19",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/Which-countries-are-restricted-from-using-Canvas/ta-p/27",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-does-Canvas-define-the-terms-used-to-describe-its-features/ta-p/51",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Admin-role/ta-p/78",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Designer-role/ta-p/14",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Teacher-role/ta-p/86",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Teacher-Assistant-role/ta-p/84",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Student-role/ta-p/71",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Observer-role/ta-p/4",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-term-dates-course-dates-and-section-dates-work-in-Canvas/ta-p/37",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-tools-and-computer-skills-do-I-need-to-use-Canvas/ta-p/446129",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-enable-third-party-cookies-in-my-browser/ta-p/605670",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-set-up-the-Lucid-Education-Suite-in-my-course/ta-p/618243",
    # Canvas Features
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-Course-Analytics/ta-p/73",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Announcements/ta-p/39",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Assignments/ta-p/9",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-Assignment-Enhancements/ta-p/416679",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Calendar/ta-p/76",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-Chat/ta-p/63",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Collaborations/ta-p/61",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Conferences/ta-p/53",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Course-Import-Tool/ta-p/21",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Discussions/ta-p/3",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Discussion-Summaries/ta-p/654034",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-DocViewer/ta-p/647614",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-ePortfolios/ta-p/1",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-External-Apps-LTI-Tools/ta-p/57",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Files/ta-p/7",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Grades-and-the-Gradebook/ta-p/49",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-grading-schemes/ta-p/41",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-What-If-Grades/ta-p/25",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Groups/ta-p/16",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Inbox/ta-p/55",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Mastery-Paths/ta-p/404483",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Modules/ta-p/6",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Outcomes/ta-p/75",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Pages/ta-p/5",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Peer-Reviews/ta-p/615313",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Profile-Settings/ta-p/15",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-question-banks/ta-p/23",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Quizzes/ta-p/68",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-New-Quizzes/ta-p/17",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Rich-Content-Editor-RCE/ta-p/12",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Roll-Call-Attendance-Tool/ta-p/59",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Rubrics/ta-p/35",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Scheduler/ta-p/11",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-SIS-Imports/ta-p/47",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-Smart-Search/ta-p/652958",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-SpeedGrader/ta-p/13",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-User-Settings/ta-p/647506",
    # Canvas Help
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-Canvas-Welcome-Tour/ta-p/618756",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-Training-Services-Portal/ta-p/618752",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-get-help-with-Canvas/ta-p/618758",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-contact-Canvas-Support/ta-p/389767",
    # Canvas Mobile
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-can-I-use-Canvas-on-my-mobile-device/ta-p/615335",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-access-Canvas-using-a-mobile-browser-on-my-Android/ta-p/65",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-access-Canvas-using-a-mobile-browser-on-my-iOS-device/ta-p/8",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Canvas-Student-app/ta-p/31",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Canvas-Teacher-app/ta-p/69",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Canvas-Parent-app/ta-p/10",
    # Calendar
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-Calendar/ta-p/617613",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-an-event-to-my-personal-calendar/ta-p/617612",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-an-event-to-a-course-calendar/ta-p/617610",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-an-assignment-from-the-Calendar/ta-p/617606",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-a-to-do-item-in-the-Calendar/ta-p/617599",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-a-recurring-event-in-a-calendar/ta-p/617604",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-edit-or-delete-an-event-or-assignment-in-the-Calendar/ta-p/617605",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-filter-the-Calendar-view-by-course/ta-p/617609",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-set-a-different-event-date-for-each-section-in-a-course/ta-p/617603",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-and-duplicate-an-event-for-course-sections-in-the/ta-p/617611",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-subscribe-to-the-Calendar-feed-using-Google-Calendar/ta-p/617614",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-an-appointment-group-in-a-course-calendar/ta-p/617602",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-or-edit-an-appointment-group-in-a-course-calendar/ta-p/617608",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-sign-up-for-an-appointment-in-the-Calendar/ta-p/617615",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-subscribe-to-the-Calendar-feed-using-Outlook-com/ta-p/617601",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-a-group-calendar/ta-p/617600",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-and-manage-Account-Calendars/ta-p/617598",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-the-Calendar-iCal-feed-to-subscribe-to-an-external/ta-p/617607",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-an-event-to-the-schedule-in-Canvas-for-Elementary/ta-p/644985",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-mark-an-assignment-or-calendar-event-with-an-important/ta-p/644976",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-my-schedule-as-a-student-in-Canvas-for-Elementary/ta-p/644981",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-the-schedule-for-a-student-as-an-observer-in/ta-p/644984",
    # Dashboard
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-Dashboard-as-a-student/ta-p/618762",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-Dashboard-as-an-instructor/ta-p/618754",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-Dashboard-as-an-observer/ta-p/618747",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-my-courses-in-the-Card-View-Dashboard/ta-p/618765",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-grades-for-all-courses-in-the-Card-View-Dashboard/ta-p/618748",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-global-activity-for-all-courses-in-the-Recent/ta-p/618764",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-To-Do-list-and-sidebar-in-the-Dashboard-as-a/ta-p/618761",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-To-Do-list-for-all-my-courses-in-the-List-View/ta-p/618751",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-a-To-Do-item-in-the-List-View-Dashboard-as-a/ta-p/618755",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-To-Do-list-and-sidebar-in-the-Dashboard-as-an/ta-p/618753",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-share-feedback-using-Instructure-s-Net-Promoter-Score/ta-p/646731",
    # ePortfolios
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-create-a-new-ePortfolio/ta-p/616171",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-ePortfolio-Getting-Started-Wizard/ta-p/616178",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-organize-my-ePortfolio/ta-p/616177",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-create-a-new-ePortfolio-section/ta-p/616175",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-organize-my-ePortfolio-pages/ta-p/616176",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-edit-the-default-page-in-my-ePortfolio-section/ta-p/616172",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-create-a-new-page-in-my-ePortfolio-section/ta-p/616173",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-preview-a-page-in-my-ePortfolio/ta-p/616179",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-rich-text-content-to-my-ePortfolio-page/ta-p/616181",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-HTML-embedded-content-to-my-ePortfolio-page/ta-p/616165",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-retrieve-assignments-from-previous-courses-in-my/ta-p/616167",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-upload-a-file-or-image-to-my-ePortfolio-page/ta-p/616168",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-enable-comments-in-my-ePortfolio/ta-p/616166",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-download-the-contents-of-my-ePortfolio/ta-p/616170",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-can-I-share-a-private-ePortfolio/ta-p/616169",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-make-my-ePortfolio-public/ta-p/616174",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-delete-my-ePortfolio/ta-p/616180",
    # Files
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-Files/ta-p/617201",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-course-files/ta-p/617226",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-group-files/ta-p/617203",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-my-user-files/ta-p/617227",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-preview-a-file-in-Files/ta-p/617193",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-move-and-organize-files/ta-p/617197",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-create-a-folder-in-Files/ta-p/617206",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-delete-a-file-or-folder-in-Files/ta-p/617196",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-download-a-file-or-folder/ta-p/617195",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-edit-a-file-in-Files/ta-p/617205",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-rename-a-file-or-folder/ta-p/617200",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-restrict-files-and-folders/ta-p/617189",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-set-usage-rights-and-user-access-for-a-file-or-folder/ta-p/617208",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-upload-a-file-in-Files/ta-p/617194",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-upload-ZIP-files-in-Files/ta-p/617192",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-bulk-upload-files/ta-p/617198",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-my-Google-Drive-files-in-Canvas/ta-p/617188",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-my-Microsoft-Office-365-files-in-Canvas/ta-p/617186",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-types-of-files-can-I-preview-in-Canvas/ta-p/617187",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-types-of-media-files-can-I-upload-in-Canvas/ta-p/617185",
    # Global Navigation
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-log-in-to-Canvas/ta-p/618760",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-Global-Navigation-Menu/ta-p/618767",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-the-Global-Announcements-page/ta-p/618749",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-my-recent-page-view-history-in-Canvas/ta-p/618750",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-log-out-of-Canvas/ta-p/618763",
    # Inbox
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-Inbox/ta-p/616671",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-send-a-message-to-a-user-in-a-course-in-the-Inbox/ta-p/616669",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-send-a-message-to-all-course-users-in-the-Inbox/ta-p/616659",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-find-my-unread-messages-in-the-Inbox/ta-p/616660",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-send-a-message-to-a-user-in-a-student-group-in-the/ta-p/616657",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-attach-a-file-to-a-message-in-the-Inbox/ta-p/616658",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-record-or-upload-media-to-a-message-in-the-Inbox/ta-p/616664",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-reply-to-a-message-in-the-Inbox/ta-p/616662",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-privately-reply-to-one-individual-in-a-group-message-in/ta-p/616663",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-reply-to-submission-comments-in-the-Inbox/ta-p/616667",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-forward-a-message-from-a-conversation-in-the-Inbox/ta-p/616668",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-delete-a-message-from-a-conversation-in-the-Inbox/ta-p/616665",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-archive-a-conversation-in-the-Inbox/ta-p/616677",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-new-people-to-an-existing-conversation-in-the-Inbox/ta-p/616670",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-delete-a-conversation-in-the-Inbox/ta-p/616672",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-filter-conversations-in-the-Inbox/ta-p/616656",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-star-a-conversation-in-the-Inbox/ta-p/616655",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-manage-my-Inbox-settings/ta-p/616661",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-Differentiated-Tags-in-the-Inbox-as-an-instructor/ta-p/657552",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-translate-an-Inbox-Message-using-AI-Translations/ta-p/660445",
    # Profile and User Settings
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-a-profile-picture-in-my-user-account/ta-p/615332",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-contact-methods-to-receive-Canvas-notifications/ta-p/615331",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-an-additional-email-address-as-a-contact-method-in/ta-p/615316",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-a-text-SMS-contact-method-in-Canvas/ta-p/615315",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-change-my-default-email-address-in-my-user-account/ta-p/615314",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-change-my-login-password-in-my-user-account/ta-p/615323",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-change-the-settings-in-my-user-account/ta-p/615322",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-edit-my-profile-in-my-user-account/ta-p/615501",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-change-the-language-preference-in-my-user-account/ta-p/615328",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-manage-my-Canvas-notification-settings/ta-p/615326",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-course-content-offline-as-an-ePub-file/ta-p/615317",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-set-a-time-zone-in-my-user-account/ta-p/615318",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-merge-my-user-account-with-an-account-using-the-same/ta-p/615319",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-manage-feature-options-in-my-user-account/ta-p/615327",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-feature-options-can-I-enable-in-my-user-account/ta-p/615324",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-set-up-multi-factor-authentication-for-my-user-account/ta-p/615333",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-manage-API-access-tokens-in-my-user-account/ta-p/615312",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-Slack-as-a-contact-method-in-Canvas/ta-p/615329",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-select-personal-pronouns-in-my-user-account/ta-p/615336",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-a-QR-code-to-log-in-to-the-Canvas-mobile-apps/ta-p/615320",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-manage-notifications-for-a-single-course/ta-p/615325",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-enable-the-high-contrast-user-interface-in-Canvas/ta-p/615334",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-manage-confetti-animations-in-Canvas/ta-p/615330",
    # Rich Content Editor (RCE)
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-access-the-Rich-Content-Editor/ta-p/618248",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-and-modify-text-in-the-Rich-Content-Editor/ta-p/618221",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-menubar-in-the-Rich-Content-Editor/ta-p/618230",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-insert-a-table-using-the-Rich-Content-Editor/ta-p/618220",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-HTML-view-in-the-Rich-Content-Editor/ta-p/618225",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-resize-the-Rich-Content-Editor/ta-p/618246",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-create-hyperlinks-to-external-URLs-in-the-Rich-Content/ta-p/618224",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-create-hyperlinks-to-user-group-or-course-files-in-the/ta-p/618223",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-create-hyperlinks-to-course-or-group-content-in-the/ta-p/618247",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-edit-or-remove-hyperlinks-in-the-Rich-Content-Editor/ta-p/618222",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-upload-and-embed-an-image-in-the-Rich-Content-Editor/ta-p/618228",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-embed-images-from-Canvas-into-the-Rich-Content-Editor/ta-p/618227",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-upload-and-embed-a-media-file-from-my-computer-in-the/ta-p/618237",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-manage-alt-text-and-display-options-for-images-embedded/ta-p/618226",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-upload-a-document-in-the-Rich-Content-Editor/ta-p/618233",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-embed-a-file-from-Google-Drive-into-the-Rich-Content/ta-p/618241",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-create-a-hyperlink-from-Microsoft-Office-365-in-the/ta-p/618240",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-link-to-or-embed-a-YouTube-video-in-the-Rich-Content/ta-p/618239",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-embed-media-files-from-Canvas-into-the-Rich-Content/ta-p/618242",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-link-to-a-document-from-Canvas-in-the-Rich-Content/ta-p/618244",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-math-Equation-Editor-in-the-Rich-Content-Editor/ta-p/618231",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-record-media-using-the-Rich-Content-Editor/ta-p/618235",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-add-captions-to-new-or-uploaded-videos-in-the-Rich/ta-p/618250",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-view-captions-in-a-video-in-the-Rich-Content-Editor/ta-p/618249",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-manage-video-options-in-the-Rich-Content-Editor/ta-p/618218",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-select-content-from-an-external-app-in-the-Rich-Content/ta-p/618219",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-embed-media-from-an-external-source-in-the-Rich-Content/ta-p/618217",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-embed-a-Lucid-document-in-the-Rich-Content-Editor/ta-p/618229",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-use-the-Accessibility-Checker-in-the-Rich-Content/ta-p/618238",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-create-an-icon-using-the-Rich-Content-Editor-Icon-Maker/ta-p/618234",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-edit-an-icon-made-using-the-Rich-Content-Editor-Icon/ta-p/618236",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-delete-an-icon-made-using-the-Rich-Content-Editor-Icon/ta-p/618232",
    # Web Services
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-connect-to-web-services-outside-of-Canvas/ta-p/617184",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-connect-to-Google-Drive-as-a-web-service-in-Canvas/ta-p/617183",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/How-do-I-connect-to-Skype-as-a-web-service-in-Canvas/ta-p/617190",
    # Canvas for Elementary
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/Canvas-for-Elementary-FAQ/ta-p/644982",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-Canvas-for-Elementary/ta-p/644983",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Homeroom-in-Canvas-for-Elementary/ta-p/644980",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-a-subject-in-Canvas-for-Elementary/ta-p/644978",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-is-the-Schedule-in-Canvas-for-Elementary/ta-p/644977",
    "https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-Important-Dates-in-Canvas-for-Elementary/ta-p/644979",
]

In [ ]:
! pip3 install beautifulsoup4

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
import bs4

In [24]:
class ContextGenerator:
     
    def __init__(self, sources: list[str], chunk_size=4200, chunk_overlap=200):
        self._sources = sources
        self._chunk_size = chunk_size
        self._chunk_overlap = chunk_overlap
        self._text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len)
        self._total_chunks = 0
    
    def _clean_page_content(self, document):
        """Clean video transcript and reference labels"""

        matches = re.findall('[0-9]{2}:[0-9]{2}: [0-9A-Za-z ;.,!?-]*',document.page_content)

        if not matches:
            return document

        page_content = document.page_content
        idx1 = page_content.find(matches[0])
        idx2 = page_content.find(matches[-1]) + len(matches[-1])
        page_content = page_content[:idx1] + page_content[idx2:]
        page_content = re.sub(r'\[\d+\]','',page_content)
        page_content = re.sub('\n',' ',page_content)
        document.page_content = page_content
        
        return document
    
    def _populate_metadata(self, document):

        """Add doc_id and doc_title to metadata"""

        source = document.metadata.get('source')
        document.metadata['doc_id'] = self._sources.index(source)
        document.metadata['doc_title'] = " ".join(source.split("/")[-3].split("-"))

        return document
    
    def _prepend_doc_title(self, document):
        
        """Prepend chunk page_content with parent doc_tile"""

        doc_title = document.metadata.get('doc_title')
        document.page_content = f"Document Title: {doc_title}\n\n" + document.page_content

        return document
    
    def load_contexts(self):

        """Load, clean, split documents from web sources"""

        loader = WebBaseLoader(
            web_paths= self._sources,
            bs_kwargs={
                "parse_only": bs4.SoupStrainer(id="content"),
            },
            bs_get_text_kwargs={"separator": "\n", "strip": True},
        )

        contexts = []

        for doc in loader.lazy_load():
            cleaned_doc = self._clean_page_content(doc)
            cleaned_doc_with_metadata = self._populate_metadata(cleaned_doc)
            chunks = list(map(self._prepend_doc_title, self._text_splitter.split_documents([cleaned_doc_with_metadata])))
            chunks = list(map(lambda doc: doc.page_content.strip(), chunks))
            self._total_chunks += len(chunks)
            contexts.append(chunks)
        
        return contexts

In [25]:
contexts = ContextGenerator(canvas_basics_guide).load_contexts()

In [26]:
len(contexts)

229

In [27]:
greater_than_2 = 0
greater_than_3 = 0
for c in contexts:
    if len(c) > 2:
        greater_than_2 += 1
    if len(c) > 3:
        greater_than_3 += 1

In [28]:
print(f"Documents with more than 2 chunks: {greater_than_2}")
print(f"Documents with more than 3 chunks: {greater_than_3}")

Documents with more than 2 chunks: 4
Documents with more than 3 chunks: 1


In [29]:
! pip3 install ipywidgets

In [30]:
synthesizer.generate_goldens_from_contexts(
    contexts=contexts,
    include_expected_output=True,
    max_goldens_per_context=2,
) # more params to explore


Output()

💰 API cost: 10.313687

[Golden(input='In the Canvas web interface and in the Teacher/Student/Parent mobile apps, where can I find and access course Modules?', actual_output=None, expected_output='Canvas Web Interface  \n1. Log in to Canvas and open your course.  \n2. In the left-hand course navigation menu, click Modules.  \n3. Expand or select any module to view its content.  \n\nCanvas Mobile Apps (Teacher/Student/Parent)  \n1. Open the Canvas app and tap the course card.  \n2. If you don’t see Modules, tap the navigation icon (☰) to open the course menu.  \n3. Tap Modules to access and browse your course modules.', context=["Document Title: What is Canvas\n\nWhat is Canvas?\nCanvas is a web-based learning management system, or LMS. It is used by learning institutions, educators, and students to access and manage online course learning materials and communicate about skill development and learning achievement.\nCanvas includes a variety of customizable course creation and management tools, course and user 

In [ ]:
synthesizer.save_as(
	file_type="csv",
	file_name="synthetic_dataset-basics-guide",
	directory="../data/datasets"
)

Synthetic goldens saved at ./synthetic_dataset-large.csv!

'./synthetic_dataset-large.csv'

In [ ]:
# dataframe = synthesizer.to_pandas()

In [ ]:
# dataframeø